In [1]:
import vk_api

from vk_api.longpoll import VkLongPoll, VkEventType
import os.path
from time import time,sleep
import numpy as np
import pandas as pd
import json
from random import randint
from photo_functions import main_photo_id, load_embeddings_1_user_2_photos,find_closest_by_2_photos,\
recommend_1_photo, write_msg,msg_recomendation,get_main_id_num
from photo_dictionaries import message_all_func, keyboard_all_func, keyboard_to_photo, \
keyboard_what_to_write_next_5_or_back, keyboard_back, keyboard_next_5_or_back, keyboard_what_to_write,keyboard_back_error,\
message_2_1, message_ask_photo, message_what_to_write, message_end

global avg_score
global write_msg


# API-ключ созданный ранее
token = "c900bb4d9d7657a238e8580eeb91aba9f22df0a811b435bcccfb806bd214c166b845fbf7b9f8f787b17df"
# Авторизуемся как сообщество
vk_group = vk_api.VkApi(token=token)

# Работа с сообщениями
longpoll = VkLongPoll(vk_group)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
Model filename: 20170512-110547/20170512-110547.pb
Instructions for updating:
Use tf.gfile.GFile.


In [2]:
dir_for_check=['Bot_Dialog_History','Closest_Photo_Recommedations','Precalculated_Embeddings','recomendations']
for d in dir_for_check:
    directory=os.getcwd()+'\\Data\\'+d
    try:
        os.stat(directory)
    except:
        os.mkdir(directory)    

In [3]:
%run recomendation_profile/recomendation_profile.py

In [ ]:
path_score_2_users = path_to_save + '\\score_2_users.pkl'
path_avg_score=path_to_save + '\\avg_score.pkl'
###
if os.path.exists(path_avg_score):
    avg_score = pd.read_pickle(path_avg_score)
else:
    avg_score=pd.DataFrame({'id_user':[], 'avg_score': []})
    avg_score.to_pickle(path_avg_score)
    
if os.path.exists(path_score_2_users):
    user_score_2_users = pd.read_pickle(path_score_2_users)
else:
    user_score_2_users=pd.DataFrame({'id_user': [], 'id_target':[], 'score': []})
    user_score_2_users.to_pickle(path_score_2_users)

In [ ]:
dont_stop = True

while dont_stop:
    try:
        # Основной цикл
        for event in longpoll.listen():

            # Если пришло новое сообщение
            if event.type == VkEventType.MESSAGE_NEW:

                # Если это входящее сообщение
                if event.to_me:

                    ev_user_id = event.user_id

                    dialog_history_path = 'Data/Bot_Dialog_History/' + str(ev_user_id) + '.pkl'

                    if os.path.exists(dialog_history_path):
                        dialog_history = pd.read_pickle(dialog_history_path)
                    else:
                        # Если нет файла истории коммуникаций, заводим его
                        dialog_history = pd.DataFrame({'id':[ev_user_id],'time':[int(time())],
                                                       'state':['0'],'message_from_u':[''],
                                                       'meassage_to_u':[''],'target_id':[np.nan]})
                        dialog_history.to_pickle(dialog_history_path)

                    user_id = ev_user_id

                    dialog_history_path = 'Data/Bot_Dialog_History/' + str(user_id) + '.pkl'
                    dialog_history = pd.read_pickle(dialog_history_path)

                    state = dialog_history.state.values[-1] 
                    user_text = event.text # Сообщение от пользователя

                    print(user_text)

                    if state=='0':

                        write_msg(user_id,message_all_func,keyboard_all_func)

                        to_history = pd.DataFrame({'id':[user_id],'time':[int(time())],
                                                       'state':['1'],'message_from_u':[user_text],
                                                       'meassage_to_u':['Базовое меню.'],'target_id':[np.nan]})  

                        dialog_history = pd.concat([dialog_history,to_history])
                        dialog_history.to_pickle(dialog_history_path)

                    elif state=='1':

                        if user_text=='Для отношений':
                            target_id=msg_recomendation (user_id,keyboard_what_to_write_next_5_or_back,recomendation_profile_predict,avg_score)

                            #write_msg(user_id,recomendation_profile(user_id),keyboard_what_to_write_next_5_or_back)
                            to_history = pd.DataFrame({'id':[user_id],'time':[int(time())],
                                                           'state':['2_1'],'message_from_u':[user_text],
                                                           'meassage_to_u':['Рекомендация Девушки'],'target_id':[np.nan]})

                        elif user_text=='По фото':

                            write_msg(user_id,message_ask_photo,keyboard_back)

                            to_history = pd.DataFrame({'id':[user_id],'time':[int(time())],
                                                           'state':['3_1'],'message_from_u':[user_text],
                                                           'meassage_to_u':['Просьба отправить фото.'],'target_id':[np.nan]}) 
                        elif user_text=='Вероятность отношений':

                            write_msg(user_id,message_ask_photo,keyboard_back)

                            to_history = pd.DataFrame({'id':[user_id],'time':[int(time())],
                                                           'state':['4_1'],'message_from_u':[user_text],
                                                           'meassage_to_u':['Просьба отправить фото.'],'target_id':[np.nan]}) 


                        elif user_text=='Выход':

                            write_msg(user_id,message_end)

                            to_history = pd.DataFrame({'id':[user_id],'time':[int(time())],
                                                           'state':['0'],'message_from_u':[user_text],
                                                           'meassage_to_u':['Прощание.'],'target_id':[np.nan]}) 

                        else:

                            write_msg(user_id,'Пожалуйста, нажми на одну из кнопок ниже.',keyboard_all_func)

                            to_history = pd.DataFrame({'id':[user_id],'time':[int(time())],
                                                           'state':['0'],'message_from_u':[user_text],
                                                           'meassage_to_u':['Просьба нажать на кнопки ниже.'],'target_id':[np.nan]}) 

                        dialog_history = pd.concat([dialog_history,to_history])
                        dialog_history.to_pickle(dialog_history_path)

                    elif state=='2_1':

                        if user_text=="Не нравится":
                            to_history = pd.DataFrame({'id':[user_id],'time':[int(time())],
                                                       'state':['2_1'],'message_from_u':[user_text],
                                                       'meassage_to_u':['Не_нравится'],'target_id':[target_id]})

                            target_id=msg_recomendation (user_id,keyboard_what_to_write_next_5_or_back,recomendation_profile_predict,avg_score)



                        elif user_text=="Нравится": 
                            to_history = pd.DataFrame({'id':[user_id],'time':[int(time())],
                                                       'state':['2_1'],'message_from_u':[user_text],
                                                       'meassage_to_u':['Нравится'],'target_id':[target_id]})

                            target_id=msg_recomendation (user_id,keyboard_what_to_write_next_5_or_back,recomendation_profile_predict,avg_score)



                        elif user_text=="Назад":

                            write_msg(user_id,message_all_func,keyboard_all_func)
                            to_history = pd.DataFrame({'id':[user_id],'time':[int(time())],
                                                           'state':['1'],'message_from_u':[user_text],
                                                           'meassage_to_u':['Базовое меню.'],'target_id':[np.nan]})  

                        dialog_history = pd.concat([dialog_history,to_history])
                        dialog_history.to_pickle(dialog_history_path)

                    elif state=='3_1':

                        target_id = user_text.split('/')[-1]

                        cl_photo_path = 'Data/Closest_Photo_Recommedations/' + str(ev_user_id) + '_' + target_id + '.pkl'

                        if os.path.exists(cl_photo_path):
                            # TODO : не рекомендовать для целевой девушки её же профиль
                            # TODO : из закрытого профайла прямым линком скачивать фото
                            to_history = recommend_1_photo(cl_photo_path,user_id,target_id,user_text,keyboard_to_photo)

                        elif user_text=="Назад":

                            write_msg(user_id,message_all_func,keyboard_all_func)
                            to_history = pd.DataFrame({'id':[user_id],'time':[int(time())],
                                                           'state':['1'],'message_from_u':[user_text],
                                                           'meassage_to_u':['Базовое меню.'],'target_id':[np.nan]})

                        else:
                            try:
                                target_df = load_embeddings_1_user_2_photos(target_id)
                                if (target_df.len_1[0]==0) and (target_df.len_2[0]!=1):
                                    write_msg(user_id,'Мы не смогли распознать лица на главном фото, пожалуйста, пришли другую ссылку.',keyboard_back)
                                    to_history = pd.DataFrame({'id':[user_id],'time':[int(time())],
                                       'state':['3_1'],'message_from_u':[user_text],
                                       'meassage_to_u':['Не смогли распознать лица'],'target_id':[np.nan]})

                                elif target_df.len_1[0]>1 and target_df.len_2[0]!=1:
                                    write_msg(user_id,'На главном фото видно более одного лица, пожалуйста, пришли другую ссылку.',keyboard_back)
                                    to_history = pd.DataFrame({'id':[user_id],'time':[int(time())],
                                       'state':['3_1'],'message_from_u':[user_text],
                                       'meassage_to_u':['На фото более одного лица'],'target_id':[np.nan]})

                                else:
                                    write_msg(user_id,'Подожди пожалуйста, находим наиболее похожих на эту девушку по фото, \
                                                       это займет 5-10 секунд.')
                                    find_closest_by_2_photos(user_id,target_df).to_pickle(cl_photo_path)
                                    to_history = recommend_1_photo(cl_photo_path,user_id,target_id,user_text,keyboard_to_photo)

                                    #TODO описать ошибку, когда закончится пул девушек

                            except:

                                write_msg(user_id,'К сожалению, мы не смогли распознать ссылку. Пришли, пожалуйста, другую ссылку.',keyboard_back)
                                #TODO сделать кнопку назад

                                to_history = pd.DataFrame({'id':[user_id],'time':[int(time())],
                                       'state':['3_1'],'message_from_u':[user_text],
                                       'meassage_to_u':['Не смогли распознать ссылку'],'target_id':[np.nan]}) 

                        dialog_history = pd.concat([dialog_history,to_history])
                        dialog_history.to_pickle(dialog_history_path)

                    elif state=='3_2': 

                        target_id = dialog_history.target_id.values[-1] # Берем целевую девушку из предыдущего шага

                        if user_text=="1. Что написать?":

                            #TODO к сообщению еще раз приложить ссылку на профиль девушки.

                            vk_group.method('messages.send', {'user_id': user_id, 
                                                'message': message_what_to_write,
                                                'keyboard': json.dumps(keyboard_what_to_write,ensure_ascii=False)})

                            to_history = pd.DataFrame({'id':[user_id],'time':[int(time())],
                               'state':['3_2'],'message_from_u':[user_text],
                               'meassage_to_u':['Что можно написать.'],'target_id':[target_id]})

                        elif user_text=="2. Следующую девушку": 

                            cl_photo_path = 'Data/Closest_Photo_Recommedations/' + str(ev_user_id) + '_' + target_id + '.pkl'
                            to_history = recommend_1_photo(cl_photo_path,user_id,target_id,user_text,keyboard_to_photo)

                        elif user_text=="3. Похожие на другую":

                            write_msg(user_id,message_ask_photo)
                            to_history = pd.DataFrame({'id':[user_id],'time':[int(time())],
                                                           'state':['3_1'],'message_from_u':[user_text],
                                                           'meassage_to_u':['Просьба отправить фото.'],'target_id':[np.nan]}) 

                        elif user_text=="4. Главное меню":

                            write_msg(user_id,message_all_func,keyboard_all_func)
                            to_history = pd.DataFrame({'id':[user_id],'time':[int(time())],
                                                           'state':['1'],'message_from_u':[user_text],
                                                           'meassage_to_u':['Базовое меню.'],'target_id':[np.nan]})   

                        else:
                            #TODO написать что-то в случае набора текста. Пока отправляем следующую девушку.
                            cl_photo_path = 'Data/Closest_Photo_Recommedations/' + str(ev_user_id) + '_' + target_id + '.pkl'
                            to_history = recommend_1_photo(cl_photo_path,user_id,target_id,user_text,keyboard_to_photo)

                        dialog_history = pd.concat([dialog_history,to_history])
                        dialog_history.to_pickle(dialog_history_path)
                    elif state=='4_1':
                        if user_text=="Назад":
                            write_msg(user_id,message_all_func,keyboard_all_func)
                            to_history = pd.DataFrame({'id':[user_id],'time':[int(time())],
                                                           'state':['1'],'message_from_u':[user_text],
                                                           'meassage_to_u':['Базовое меню.'],'target_id':[np.nan]})
                        else:
                            try:
                                target_id = user_text.split('/')[-1]
                                target_id=get_main_id_num(target_id)
                                score=recomendation_score_2_users(user_id,target_id,avg_score,user_score_2_users)
                                score=int(np.round(score*100,0))
                                if score is not None:
                                    write_msg(user_id,"Вероятность отношений с этой девушкой составляет "+ str(score)+" процентов",keyboard_back)
                                    to_history = pd.DataFrame({'id':[user_id],'time':[int(time())],
                                                           'state':['4_1'],'message_from_u':[user_text],
                                                           'meassage_to_u':['Вероятность отношений'],'target_id':[target_id]})
                                else: 
                                    write_msg(user_id,'К сожалению, мы не смогли распознать ссылку либо выгрузить профиль. Пришли, пожалуйста, другую ссылку.',keyboard_back)
                                #TODO сделать кнопку назад
                                    to_history = pd.DataFrame({'id':[user_id],'time':[int(time())],
                                       'state':['4_1'],'message_from_u':[user_text],
                                       'meassage_to_u':['Не смогли распознать ссылку или профиль'],'target_id':[np.nan]}) 
                            except:
                                write_msg(user_id,'К сожалению, мы не смогли распознать ссылку либо выгрузить профиль. Пришли, пожалуйста, другую ссылку.',keyboard_back)
                                to_history = pd.DataFrame({'id':[user_id],'time':[int(time())],
                                       'state':['4_1'],'message_from_u':[user_text],
                                       'meassage_to_u':['Не смогли распознать ссылку или профиль'],'target_id':[np.nan]}) 
                        dialog_history = pd.concat([dialog_history,to_history])
                        dialog_history.to_pickle(dialog_history_path)

    except:
        sleep(60)

Привет
Для отношений
420228311
Не нравится
142483399
Нравится
60309444
Не нравится
382093321
Назад
По фото
https://vk.com/id382093321
id382093321


C:\Users\pvmek\Documents\GitHub\vk\photo_functions.py:216: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result['id'] = result['id'].astype(int).astype(str)


223985650
2. Следующую девушку
2052919
236726455
2. Следующую девушку
3736802
4. Главное меню
Вероятность отношений
https://vk.com/id3736802
Выгрузка пользователя закончена... :      1.92
Выбор девушек... :      1.94
Предсказание :      1.94
Сбор результата... :      1.97
Расчет окончен... :      1.98
https://vk.com/id236726455
Выгрузка пользователя закончена... :      1.92
Выбор девушек... :      1.92
Предсказание :      1.94
Сбор результата... :      1.95
Расчет окончен... :      1.95
https://vk.com/id223985650
Выгрузка пользователя закончена... :      1.88
Выбор девушек... :      1.88
Предсказание :      1.89
Сбор результата... :      1.91
Расчет окончен... :      1.91
ghbdtn
Вероятность отношений
Назад
Для отношений
364039720
Не нравится
82665123
Не нравится
93922610
Не нравится
91251103
Назад
По фото
